In [1]:
import numpy as np
import matplotlib as plt
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
data = mnist.data 
answer = mnist.target 

train_set, test_set, train_target, test_target = train_test_split(data, answer, test_size=10000)

In [8]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
data = mnist.data 
answer = mnist.target 

train_set, test_set, train_target, test_target = train_test_split(data, answer, test_size=10000)

/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def euclidean_distance1(x, y):
    return np.sqrt(np.sum(x ** 2, axis=1)[:,np.newaxis] + np.sum(y ** 2, axis=1)
            - 2 * np.dot(x, np.transpose(y)))


def euclidean_distance(x, y):
    return np.sqrt(np.sum((x[:, np.newaxis] - y) ** 2, axis=2))

def cosine_distance1(x, y):
   # print(x, y)
    norma = np.sqrt(np.sum(np.square(x)[:,np.newaxis], axis=1).reshape(-1, 1) * np.sum(np.square(y), axis=1)) 
    if len(np.where(norma == 0)[0]) == 0:
        return np.dot(x, np.transpose(y)) / norma
    return np.array([0])
def cosine_distance(x, y):
    ans = 1 - np.dot(x, y.T) /( np.sqrt(np.sum(x ** 2, axis=1)[:, None]) * np.sqrt(np.sum(y ** 2, axis=1)[None, :]))
    ans[np.isnan(ans)] = 0
    ans[ans == np.inf] = 0
    print(np.isnan(ans).__len__(), np.isinf(ans).sum())
    return ans

def cosine_dist2(x, y):
    first = np.sqrt(np.sum(x ** 2, axis=1))
    first[first == 0] = 0.000001
    second = np.sqrt(np.sum(y ** 2, axis=1))
    second[second == 0] = 0.000001
    
    return 1 - np.dot(x, y.T) / first[:, None] / second[None, :]

def weight(array):
    eps = 0.00001
    return 1 / (array + eps)

In [103]:
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
#import distances

class KNNClassifier:
    def __init__(self, k, strategy, metric, weights, test_block_size):
        self.k = k
        self.strategy = strategy
        self.metric = metric
        self.weights = weights
        self.test_block_size = test_block_size
        if strategy != 'my_own':
            if strategy != 'brute' and strategy != 'kd_tree' and strategy != 'ball_tree':
                raise KeyError
            # change parameters
            self.skln_knn = NearestNeighbors(
                n_neighbors=k, algorithm = strategy, metric = metric)
        
    def fit(self, x, y):
        if self.strategy != 'my_own':
            self.skln_knn.fit(x, y)
        else:
            self.train_set = x
        self.train_target = y
       
    def find_kneighbors(self, x, return_distance):
        if self.strategy == 'my_own':
            if self.metric == 'euclidean':
                ans_matrix = euclidean_distance(x, self.train_set)
            if self.metric == 'cosine':
                ans_matrix = cosine_dist2(x, self.train_set)
            
            index = np.argsort(ans_matrix, axis=1)[:,:self.k]
           
            if return_distance:
                neighbour = np.sort(ans_matrix, axis=1)[:, :self.k]

                return (neighbour, index)
            return index
        else:
            return self.skln_knn.kneighbors(x, n_neighbors=self.k, return_distance=return_distance)
        
    def block_helper(self, x):
        n_blocks = x.shape[0] // self.test_block_size if x.shape[0] % self.test_block_size == 0 else x.shape[0] // self.test_block_size + 1
        dist_answer = np.array([])
        ind_answer = np.array([])
        for q in range(n_blocks):
            begin_border = q * self.test_block_size
            end_border = (q + 1) * self.test_block_size
            if self.weights:
                dist, ind = self.find_kneighbors(x[begin_border:end_border], True)
                if(len(ind_answer) == 0):
                    dist_answer = dist
                    ind_answer = ind
                else:
                    dist_answer = np.concatenate([dist_answer, dist])
                    ind_answer = np.concatenate([ind_answer, ind])
            else:
                ind = self.find_kneighbors(x[begin_border:end_border], False)
                if(len(ind_answer) == 0):
                    ind_answer = ind
                else:
                    ind_answer = np.concatenate([ind_answer, ind])
        if self.weights:
            return (dist_answer, ind_answer)
        return ind_answer
        
        
    def predict(self, x):
        n_blocks = x.shape[0] // self.test_block_size if x.shape[0] % self.test_block_size == 0 else x.shape[0] // self.test_block_size + 1
        
        answer = np.zeros(x.shape[0]).astype(int)
        
        for q in range(n_blocks):
            begin_border = q * self.test_block_size
            end_border = (q + 1) * self.test_block_size
            if self.weights:
                dist, ind = self.find_kneighbors(x[begin_border:end_border], True)
                votes = weight(dist)
                for i, mark in enumerate(self.train_target[ind]):
                    
                    answer[i + q * self.test_block_size] = np.bincount(mark.astype(int), votes[i]).argmax()
            else:
                ind = self.find_kneighbors(x[begin_border:end_border], False)
                for i in range(self.train_target[ind]):
                    answer[i + q *self.test_block_size] = np.bincount(mark.astype(int)).argmax()
        
                
        return answer
                    


In [118]:
knn = KNNClassifier(4, 'my_own', 'euclidean', False, 10)
knn.fit(train_set, train_target)
print(knn.block_helper(test_set[:20]))


(10, 60000)
beg
[[29574 46707 46820 46860]
 [22376 49274 21541 28816]
 [15880 44733 14797 30428]
 [21844 54078 23615 32731]
 [37874 46192  5065 33567]
 [51369 13832 32731 57761]
 [11672 32646 21541 57761]
 [34078 37874   529 55081]
 [ 3754 21844 20885 34284]
 [20949 25591 11835  7759]]
fin
(10, 60000)
beg
[[46192 57290 57761 52589]
 [51369 13760  1507 54190]
 [21541 36927 13832  2589]
 [10208   302 13638 22376]
 [23615 39381 21541 49511]
 [31268 56981 34201 17650]
 [22336 11564 45279 37874]
 [24706 49706 43372 55542]
 [57761 21541  2589 34284]
 [ 7729 11340 21844 49511]]
fin
[[29574 46707 46820 46860]
 [22376 49274 21541 28816]
 [15880 44733 14797 30428]
 [21844 54078 23615 32731]
 [37874 46192  5065 33567]
 [51369 13832 32731 57761]
 [11672 32646 21541 57761]
 [34078 37874   529 55081]
 [ 3754 21844 20885 34284]
 [20949 25591 11835  7759]
 [46192 57290 57761 52589]
 [51369 13760  1507 54190]
 [21541 36927 13832  2589]
 [10208   302 13638 22376]
 [23615 39381 21541 49511]
 [31268 56981

n объектов, n_folds фолдов
Делим выборку на n_

In [6]:
#cross_validation
def accuracy(prediction, answer):
    return len(np.where(prediction == answer)[0]) / len(prediction)

def kfold(n, n_folds):
    if n_folds > n or n_folds < 2:
        raise TypeError
    
    ind_array = np.arange(n)
    if n % n_folds == 0:
        ind_array = np.split(ind_array, n_folds)
    else:
        array = np.split(ind_array[:(n % n_folds) * (n // n_folds + 1)], n % n_folds)
        array = array + np.split(ind_array[(n % n_folds) * (n // n_folds + 1):], n_folds - n % n_folds)
        ind_array = array                             
    answer = []
    for elem in ind_array:
        dop = np.concatenate([np.arange(elem[0]), np.arange(min(elem[::-1][0] + 1, n), n)])
        answer.append((dop, elem))
    return answer

def knn_cross_val_score(x, y, k_list, score, cv, **kwargs):
    if cv == None:
        cv = kfold(x.shape[0], 3)
    answer = {}
    dop_answer = np.zeros(len(cv) * len(k_list)).reshape(len(k_list), -1)
    model = KNNClassifier(k=max(k_list), **kwargs)
    for j, data in enumerate(cv):
        train_set = x[data[0]]
        train_target = y[data[0]]
        test_set = x[data[1]]
        test_target = y[data[1]]
        model.fit(train_set, train_target)
        if kwargs['weights']:
            dist, ind = model.find_kneighbors(test_set, return_distance=True)
            votes = 1 / (dist + 0.00001)
        else:
            ind = model.find_kneighbors(test_set, return_distance=False)
        for i, k in enumerate(k_list):
            sub_answer = np.zeros(test_set.shape[0])
            if kwargs['weights']:
                k_dist = dist[:, :k]
                k_votes = votes[:, :k]
            k_ind = ind[:, :k]
            if kwargs['weights']:
                for q, mark in enumerate(train_target[k_ind]):
                    sub_answer[q] = np.bincount(mark.astype(int), k_votes[q]).argmax()
                
            else:
                for q, mark in enumerate(train_target[k_ind]):
                    sub_answer[q] = np.bincount(mark.astype(int)).argmax()
                
                    
            
            dop_answer[i][j] = len(np.where(test_target == sub_answer)[0]) / len(test_target)
    for i, k in enumerate(k_list):
        answer[k] = dop_answer[i]
    return answer


Эксперименты

1. Исследование работы алгоритмов поиска ближайший соседей для евклидовой метрики

In [9]:
import random

In [123]:
train_set.shape

(60000, 784)

In [7]:
print(knn_cross_val_score(train_set, train_target, [4], 'accuracy', None,  strategy='my_own', metric='cosine', 
                    weights=False, test_block_size=100))

MemoryError: 

In [105]:

feature = np.sort(random.sample(range(test_set.shape[1]), 10))
print(feature)
new_train_set, new_test_set = train_set[:,feature], test_set[:,feature]

[  8  69 156 393 439 480 635 649 664 700]


In [11]:
%%time
knn_my_own = KNNClassifier(5, strategy='my_own', metric='euclidean', weights=False, test_block_size=10)
knn_my_own.fit(new_train_set, train_target)
knn_my_own.block_helper(new_test_set)

Wall time: 2min 22s


In [12]:
%%time
knn_brute = KNNClassifier(5, strategy='brute', metric='euclidean', weights=False, test_block_size=10)
knn_brute.fit(new_train_set, train_target)
knn_brute.find_kneighbors(new_test_set, return_distance=False)

Wall time: 1min 43s


In [13]:
%%time
knn_kd_tree = KNNClassifier(5, strategy='kd_tree', metric='euclidean', weights=False, test_block_size=10)
knn_kd_tree.fit(new_train_set, train_target)
knn_kd_tree.find_kneighbors(new_test_set, return_distance=False)

Wall time: 15.9 s


In [14]:
%%time
knn_ball_tree = KNNClassifier(5, strategy='ball_tree', metric='euclidean', weights=False, test_block_size=10)
knn_ball_tree.fit(new_train_set, train_target)
knn_ball_tree.find_kneighbors(new_test_set, return_distance=False)

Wall time: 20.7 s


cross-val for 10 features

In [15]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.30533893, 0.30513051, 0.28772877]), 2: array([0.30743851, 0.29912991, 0.2880288 ]), 3: array([0.31223755, 0.31743174, 0.29972997]), 4: array([0.32213557, 0.33633363, 0.31323132]), 5: array([0.32873425, 0.32973297, 0.32433243]), 6: array([0.33503299, 0.34023402, 0.32493249]), 7: array([0.33773245, 0.34173417, 0.32013201]), 8: array([0.33533293, 0.34743474, 0.3390339 ]), 9: array([0.34043191, 0.34443444, 0.34533453]), 10: array([0.34283143, 0.34233423, 0.3450345 ])}
Wall time: 18.6 s


In [108]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='cosine', 
                    weights=False, test_block_size=10))

{1: array([0.0959808 , 0.09390939, 0.11191119]), 2: array([0.09868026, 0.10261026, 0.11281128]), 3: array([0.10167966, 0.1020102 , 0.1080108 ]), 4: array([0.1034793 , 0.10171017, 0.10591059]), 5: array([0.09538092, 0.10141014, 0.11191119]), 6: array([0.09478104, 0.10081008, 0.11251125]), 7: array([0.09508098, 0.09960996, 0.11011101]), 8: array([0.0959808 , 0.10141014, 0.11521152]), 9: array([0.09658068, 0.09990999, 0.11281128]), 10: array([0.09658068, 0.10381038, 0.10591059])}
Wall time: 8.48 s


In [17]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.3875225 , 0.38133813, 0.38553855]), 2: array([0.39172166, 0.40834083, 0.39333933]), 3: array([0.39982004, 0.43234323, 0.41344134]), 4: array([0.42621476, 0.44494449, 0.43894389]), 5: array([0.43761248, 0.45094509, 0.44644464]), 6: array([0.44421116, 0.45664566, 0.45874587]), 7: array([0.44691062, 0.46444644, 0.4620462 ]), 8: array([0.45260948, 0.45754575, 0.46744674]), 9: array([0.45470906, 0.46444644, 0.46684668]), 10: array([0.455009  , 0.46624662, 0.46234623])}
Wall time: 3.97 s


In [18]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='cosine', 
                    weights=False, test_block_size=10))

{1: array([0.39382124, 0.38973897, 0.38373837]), 2: array([0.37732454, 0.39963996, 0.37983798]), 3: array([0.40371926, 0.41284128, 0.4380438 ]), 4: array([0.42021596, 0.44194419, 0.45184518]), 5: array([0.44991002, 0.45244524, 0.43954395]), 6: array([0.43071386, 0.43894389, 0.44524452]), 7: array([0.42831434, 0.43744374, 0.44584458]), 8: array([0.44511098, 0.44794479, 0.44974497]), 9: array([0.44961008, 0.44194419, 0.45064506]), 10: array([0.44481104, 0.44764476, 0.45244524])}
Wall time: 3.18 s


In [19]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='kd_tree', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.37642472, 0.38223822, 0.38793879]), 2: array([0.37132573, 0.40894089, 0.39153915]), 3: array([0.38962208, 0.42994299, 0.41794179]), 4: array([0.41061788, 0.44284428, 0.43834383]), 5: array([0.42291542, 0.44614461, 0.4440444 ]), 6: array([0.425015  , 0.45664566, 0.44464446]), 7: array([0.42921416, 0.46234623, 0.46054605]), 8: array([0.43671266, 0.45664566, 0.46474647]), 9: array([0.43671266, 0.46324632, 0.46234623]), 10: array([0.43881224, 0.46744674, 0.45994599])}
Wall time: 1.67 s


In [20]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='ball_tree', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.37132573, 0.3750375 , 0.39153915]), 2: array([0.37072585, 0.40474047, 0.40234023]), 3: array([0.38842232, 0.42934293, 0.41494149]), 4: array([0.40971806, 0.44584458, 0.43534353]), 5: array([0.42111578, 0.44944494, 0.43744374]), 6: array([0.42591482, 0.45634563, 0.450045  ]), 7: array([0.4265147 , 0.46354635, 0.45964596]), 8: array([0.4355129 , 0.45664566, 0.46444644]), 9: array([0.43641272, 0.46354635, 0.46174617]), 10: array([0.43821236, 0.46624662, 0.4590459 ])}
Wall time: 2.54 s


In [21]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.30533893, 0.30513051, 0.28772877]), 2: array([0.31103779, 0.31323132, 0.29612961]), 3: array([0.31973605, 0.32253225, 0.30153015]), 4: array([0.32843431, 0.33993399, 0.31773177]), 5: array([0.33053389, 0.33513351, 0.33033303]), 6: array([0.33323335, 0.34593459, 0.33033303]), 7: array([0.33833233, 0.34593459, 0.33333333]), 8: array([0.33413317, 0.34953495, 0.34773477]), 9: array([0.34133173, 0.34953495, 0.35193519]), 10: array([0.34493101, 0.34923492, 0.34863486])}
Wall time: 26.1 s


In [64]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='cosine', 
                    weights=True, test_block_size=10))

{1: array([0.089982  , 0.10861086, 0.10111011]), 2: array([0.08938212, 0.08490849, 0.08490849]), 3: array([0.08428314, 0.08130813, 0.08250825]), 4: array([0.08068386, 0.08130813, 0.07830783]), 5: array([0.0824835 , 0.0780078 , 0.07650765]), 6: array([0.08038392, 0.07590759, 0.07110711]), 7: array([0.07858428, 0.07440744, 0.06750675]), 8: array([0.07978404, 0.07140714, 0.06480648]), 9: array([0.08008398, 0.07230723, 0.0630063 ]), 10: array([0.10167966, 0.09180918, 0.06540654])}
Wall time: 18.8 s


In [23]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.3875225 , 0.38133813, 0.38553855]), 2: array([0.39532094, 0.40864086, 0.38913891]), 3: array([0.40371926, 0.42424242, 0.39933993]), 4: array([0.42261548, 0.4380438 , 0.42544254]), 5: array([0.43191362, 0.44494449, 0.4350435 ]), 6: array([0.4280144 , 0.45154515, 0.43744374]), 7: array([0.43521296, 0.4560456 , 0.44374437]), 8: array([0.43521296, 0.45544554, 0.450045  ]), 9: array([0.4385123 , 0.46234623, 0.45064506]), 10: array([0.43611278, 0.4650465 , 0.45334533])}
Wall time: 4.09 s


In [24]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='cosine', 
                    weights=True, test_block_size=10))

{1: array([0.39382124, 0.38973897, 0.38373837]), 2: array([0.38662268, 0.39243924, 0.37593759]), 3: array([0.4010198 , 0.40264026, 0.42394239]), 4: array([0.41331734, 0.42424242, 0.43264326]), 5: array([0.42981404, 0.4350435 , 0.41854185]), 6: array([0.4145171 , 0.42064206, 0.41884188]), 7: array([0.41061788, 0.42334233, 0.42244224]), 8: array([0.42531494, 0.43174317, 0.42484248]), 9: array([0.42531494, 0.43114311, 0.42784278]), 10: array([0.42411518, 0.42874287, 0.43024302])}
Wall time: 3.23 s


In [25]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='kd_tree', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.37642472, 0.38223822, 0.38793879]), 2: array([0.37522496, 0.40654065, 0.38913891]), 3: array([0.39262148, 0.42184218, 0.40474047]), 4: array([0.4085183 , 0.43864386, 0.42394239]), 5: array([0.41541692, 0.44134413, 0.43444344]), 6: array([0.41031794, 0.45244524, 0.4260426 ]), 7: array([0.41931614, 0.45484548, 0.44344434]), 8: array([0.41961608, 0.45484548, 0.4470447 ]), 9: array([0.42291542, 0.46174617, 0.44734473]), 10: array([0.42141572, 0.46474647, 0.450045  ])}
Wall time: 1.83 s


In [26]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='ball_tree', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.37132573, 0.3750375 , 0.39153915]), 2: array([0.37252549, 0.40294029, 0.39843984]), 3: array([0.3920216 , 0.42154215, 0.40324032]), 4: array([0.40731854, 0.44074407, 0.4230423 ]), 5: array([0.41421716, 0.44554455, 0.42844284]), 6: array([0.41121776, 0.45184518, 0.4320432 ]), 7: array([0.41721656, 0.45544554, 0.44224422]), 8: array([0.41871626, 0.4560456 , 0.44494449]), 9: array([0.42231554, 0.46174617, 0.44614461]), 10: array([0.42081584, 0.4650465 , 0.44884488])}
Wall time: 2.72 s


In [1]:
import numpy as np
import matplotlib as plt
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
data = mnist.data 
answer = mnist.target 

train_set, test_set, train_target, test_target = train_test_split(data, answer, test_size=10000)

import random
random.seed(19)
feature = np.sort(random.sample(range(test_set.shape[1]), 20))
print(feature)
new_train_set, new_test_set = train_set[:,feature], test_set[:,feature]

[ 44 110 123 151 204 266 267 274 296 333 335 355 402 421 523 532 541 598
 609 693]


/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from nearest_neighbors import KNNClassifier
from cross_validation import knn_cross_val_score

In [3]:
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='cosine', 
                    weights=False, test_block_size=10))

/Users/maksimkornilov/Documents/Hometask_Practicum/Big_task_1/distances.py:9: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - np.dot(x, y.T) / np.sqrt(np.sum(x ** 2, axis=1)[:, None]) / np.sqrt(np.sum(y ** 2, axis=1)[None, :])
/Users/maksimkornilov/Documents/Hometask_Practicum/Big_task_1/distances.py:9: RuntimeWarning: invalid value encountered in true_divide
  return 1 - np.dot(x, y.T) / np.sqrt(np.sum(x ** 2, axis=1)[:, None]) / np.sqrt(np.sum(y ** 2, axis=1)[None, :])


{1: array([0.14431443, 0.14041404, 0.14487103]), 2: array([0.16471647, 0.15571557, 0.16166767]), 3: array([0.16441644, 0.17551755, 0.16856629]), 4: array([0.17581758, 0.17671767, 0.16766647]), 5: array([0.19381938, 0.17641764, 0.16286743]), 6: array([0.20222022, 0.18631863, 0.17036593]), 7: array([0.20552055, 0.20132013, 0.17906419]), 8: array([0.21032103, 0.18691869, 0.18146371]), 9: array([0.21632163, 0.18781878, 0.18206359]), 10: array([0.21662166, 0.18931893, 0.18206359])}


In [4]:
%%time
knn_my_own = KNNClassifier(5, strategy='my_own', metric='euclidean', weights=False, test_block_size=10)
knn_my_own.fit(new_train_set, train_target)
knn_my_own.block_helper(new_test_set)

AttributeError: 'KNNClassifier' object has no attribute 'block_helper'

In [29]:
%%time
knn_brute = KNNClassifier(5, strategy='brute', metric='euclidean', weights=False, test_block_size=10)
knn_brute.fit(new_train_set, train_target)
knn_brute.find_kneighbors(new_test_set, return_distance=False)

Wall time: 1min 37s


In [30]:
%%time
knn_kd_tree = KNNClassifier(5, strategy='kd_tree', metric='euclidean', weights=False, test_block_size=10)
knn_kd_tree.fit(new_train_set, train_target)
knn_kd_tree.find_kneighbors(new_test_set, return_distance=False)

Wall time: 27.3 s


In [31]:
%%time
knn_ball_tree = KNNClassifier(5, strategy='ball_tree', metric='euclidean', weights=False, test_block_size=10)
knn_ball_tree.fit(new_train_set, train_target)
knn_ball_tree.find_kneighbors(new_test_set, return_distance=False)

Wall time: 55.9 s


cross-val for 20 features

In [5]:
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.16831683, 0.1470147 , 0.16076785]), 2: array([0.1710171 , 0.16591659, 0.16976605]), 3: array([0.1770177 , 0.16231623, 0.16346731]), 4: array([0.18061806, 0.17311731, 0.17786443]), 5: array([0.17851785, 0.17431743, 0.16946611]), 6: array([0.19051905, 0.17461746, 0.17156569]), 7: array([0.18871887, 0.17221722, 0.17246551]), 8: array([0.18661866, 0.17581758, 0.17276545]), 9: array([0.18721872, 0.17371737, 0.17606479]), 10: array([0.18691869, 0.18031803, 0.17876425])}


In [73]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.32123575, 0.34083408, 0.2580258 ]), 2: array([0.32243551, 0.3420342 , 0.26282628]), 3: array([0.33593281, 0.34833483, 0.33513351]), 4: array([0.34763047, 0.35583558, 0.34923492]), 5: array([0.35692861, 0.37023702, 0.35613561]), 6: array([0.35572885, 0.36513651, 0.35643564]), 7: array([0.36172765, 0.36723672, 0.36273627]), 8: array([0.36382723, 0.36813681, 0.36273627]), 9: array([0.36772645, 0.37263726, 0.36423642]), 10: array([0.37342531, 0.36813681, 0.36753675])}
Wall time: 3.96 s


In [74]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='cosine', 
                    weights=False, test_block_size=10))

{1: array([0.24925015, 0.26432643, 0.25772577]), 2: array([0.24805039, 0.26042604, 0.26582658]), 3: array([0.25404919, 0.27482748, 0.3420342 ]), 4: array([0.26904619, 0.28442844, 0.3510351 ]), 5: array([0.35332933, 0.36783678, 0.2880288 ]), 6: array([0.27204559, 0.28712871, 0.29042904]), 7: array([0.29094181, 0.28622862, 0.28712871]), 8: array([0.29664067, 0.28742874, 0.28592859]), 9: array([0.29724055, 0.29552955, 0.29552955]), 10: array([0.30713857, 0.29252925, 0.29852985])}
Wall time: 3.08 s


In [75]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='kd_tree', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.23935213, 0.33693369, 0.32133213]), 2: array([0.24865027, 0.34023402, 0.33123312]), 3: array([0.33713257, 0.34953495, 0.33753375]), 4: array([0.35122975, 0.3570357 , 0.3480348 ]), 5: array([0.35692861, 0.36753675, 0.35643564]), 6: array([0.35692861, 0.36363636, 0.3570357 ]), 7: array([0.36142771, 0.36483648, 0.36093609]), 8: array([0.36322735, 0.36513651, 0.360036  ]), 9: array([0.36532693, 0.36933693, 0.36183618]), 10: array([0.37312537, 0.36363636, 0.36573657])}
Wall time: 1.74 s


In [76]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='ball_tree', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.23785243, 0.33813381, 0.3270327 ]), 2: array([0.24625075, 0.3390339 , 0.33333333]), 3: array([0.33533293, 0.34893489, 0.33753375]), 4: array([0.34943011, 0.35343534, 0.34863486]), 5: array([0.35272945, 0.36513651, 0.35463546]), 6: array([0.35752849, 0.36333633, 0.35673567]), 7: array([0.35782843, 0.36423642, 0.36213621]), 8: array([0.36052789, 0.36273627, 0.35973597]), 9: array([0.36802639, 0.36753675, 0.36153615]), 10: array([0.37222555, 0.3630363 , 0.36393639])}
Wall time: 2.63 s


In [77]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.26754649, 0.19651965, 0.27722772]), 2: array([0.27264547, 0.20192019, 0.27992799]), 3: array([0.28374325, 0.29252925, 0.28292829]), 4: array([0.28764247, 0.22652265, 0.29042904]), 5: array([0.29214157, 0.30363036, 0.28682868]), 6: array([0.29574085, 0.30363036, 0.29492949]), 7: array([0.29124175, 0.30093009, 0.29912991]), 8: array([0.29124175, 0.30423042, 0.30123012]), 9: array([0.29304139, 0.30693069, 0.30243024]), 10: array([0.29244151, 0.30723072, 0.30033003])}
Wall time: 25.4 s


In [78]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='cosine', 
                    weights=True, test_block_size=10))

{1: array([0.09118176, 0.11371137, 0.10231023]), 2: array([0.08368326, 0.10321032, 0.08730873]), 3: array([0.0779844 , 0.08670867, 0.08280828]), 4: array([0.0854829 , 0.08820882, 0.08730873]), 5: array([0.09238152, 0.08670867, 0.09210921]), 6: array([0.089982  , 0.08640864, 0.09330933]), 7: array([0.09118176, 0.08730873, 0.09210921]), 8: array([0.08758248, 0.08430843, 0.09180918]), 9: array([0.08728254, 0.07980798, 0.09030903]), 10: array([0.16196761, 0.15991599, 0.090009  ])}
Wall time: 17.7 s


In [79]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.32123575, 0.34083408, 0.2580258 ]), 2: array([0.31613677, 0.33843384, 0.25772577]), 3: array([0.33203359, 0.34473447, 0.33813381]), 4: array([0.34763047, 0.35223522, 0.3510351 ]), 5: array([0.35572885, 0.35913591, 0.35313531]), 6: array([0.35632873, 0.36333633, 0.35793579]), 7: array([0.36412717, 0.36183618, 0.36543654]), 8: array([0.36292741, 0.36273627, 0.3660366 ]), 9: array([0.36712657, 0.36363636, 0.36753675]), 10: array([0.36982603, 0.36243624, 0.37023702])}
Wall time: 4.13 s


In [80]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='cosine', 
                    weights=True, test_block_size=10))

{1: array([0.24925015, 0.26432643, 0.25772577]), 2: array([0.24205159, 0.2580258 , 0.2580258 ]), 3: array([0.24715057, 0.26762676, 0.33633363]), 4: array([0.25644871, 0.27542754, 0.35193519]), 5: array([0.33953209, 0.35133513, 0.27842784]), 6: array([0.25644871, 0.27272727, 0.28172817]), 7: array([0.27234553, 0.2760276 , 0.28742874]), 8: array([0.27534493, 0.27512751, 0.28622862]), 9: array([0.27654469, 0.27752775, 0.29072907]), 10: array([0.27744451, 0.2790279 , 0.28982898])}
Wall time: 3.14 s


In [81]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='kd_tree', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.23935213, 0.33693369, 0.32133213]), 2: array([0.24205159, 0.33783378, 0.32763276]), 3: array([0.33233353, 0.34653465, 0.33873387]), 4: array([0.34943011, 0.35253525, 0.35043504]), 5: array([0.35752849, 0.35943594, 0.35313531]), 6: array([0.35842831, 0.36093609, 0.35613561]), 7: array([0.36412717, 0.360036  , 0.36243624]), 8: array([0.36232753, 0.35883588, 0.3630363 ]), 9: array([0.36622675, 0.360036  , 0.36573657]), 10: array([0.36952609, 0.35853585, 0.3690369 ])}
Wall time: 1.75 s


In [82]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='ball_tree', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.23785243, 0.33813381, 0.3270327 ]), 2: array([0.24055189, 0.33633363, 0.32793279]), 3: array([0.33053389, 0.34413441, 0.33963396]), 4: array([0.34793041, 0.34923492, 0.35163516]), 5: array([0.35302939, 0.35553555, 0.35283528]), 6: array([0.35722855, 0.36153615, 0.35553555]), 7: array([0.36142771, 0.35943594, 0.36273627]), 8: array([0.36082783, 0.35793579, 0.36273627]), 9: array([0.36742651, 0.35973597, 0.36543654]), 10: array([0.36952609, 0.35913591, 0.36873687])}
Wall time: 2.74 s


In [6]:
seed(19)
feature = np.sort(random.sample(range(test_set.shape[1]), 100))
print(feature)
new_train_set, new_test_set = train_set[:,feature], test_set[:,feature]

[  2   7  16  17  23  26  46  68  72  75  77  79  87  93 100 101 103 107
 110 116 122 131 137 141 162 175 179 187 196 206 207 212 214 238 250 269
 281 290 300 306 308 315 318 377 385 397 399 405 413 416 424 430 433 438
 447 449 450 460 462 465 466 469 472 474 477 479 487 491 505 512 514 517
 532 554 555 558 560 582 584 590 592 608 632 641 652 664 668 672 674 692
 693 719 722 736 743 745 748 751 776 783]


In [84]:
%%time
knn_my_own = KNNClassifier(5, strategy='my_own', metric='euclidean', weights=False, test_block_size=10)
knn_my_own.fit(new_train_set, train_target)
knn_my_own.block_helper(new_test_set)


Wall time: 8min 45s


In [85]:
%%time
knn_brute = KNNClassifier(5, strategy='brute', metric='euclidean', weights=False, test_block_size=100)
knn_brute.fit(new_train_set, train_target)
knn_brute.find_kneighbors(new_test_set, return_distance=False)

Wall time: 2min


In [86]:
%%time
knn_kd_tree = KNNClassifier(5, strategy='kd_tree', metric='euclidean', weights=False, test_block_size=100)
knn_kd_tree.fit(new_train_set, train_target)
knn_kd_tree.find_kneighbors(new_test_set, return_distance=False)

Wall time: 3min 48s


In [87]:
%%time
knn_ball_tree = KNNClassifier(5, strategy='ball_tree', metric='euclidean', weights=False, test_block_size=100)
knn_ball_tree.fit(new_train_set, train_target)
knn_ball_tree.find_kneighbors(new_test_set, return_distance=False)

Wall time: 3min 10s


In [9]:
new_test_set.shape, test_target.shape

((10000, 100), (10000,))

cross-val for 100 features

In [ ]:
%%time
print(knn_cross_val_score(new_train_set, train_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='euclidean', 
                    weights=False, test_block_size=10))

In [ ]:
%%time
print(knn_cross_val_score(new_train_set, train_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='cosine', 
                    weights=False, test_block_size=10))

In [90]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.83803239, 0.83978398, 0.85358536]), 2: array([0.82543491, 0.82508251, 0.83858386]), 3: array([0.84853029, 0.84488449, 0.85508551]), 4: array([0.84913017, 0.84638464, 0.86378638]), 5: array([0.85122975, 0.84008401, 0.86648665]), 6: array([0.85302939, 0.84578458, 0.86228623]), 7: array([0.85722855, 0.84248425, 0.86618662]), 8: array([0.84973005, 0.84668467, 0.86348635]), 9: array([0.84673065, 0.84128413, 0.86528653]), 10: array([0.84883023, 0.83618362, 0.86168617])}
Wall time: 4.42 s


In [91]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='cosine', 
                    weights=False, test_block_size=10))

{1: array([0.85272945, 0.85208521, 0.86018602]), 2: array([0.85062987, 0.8409841 , 0.85418542]), 3: array([0.87492501, 0.85988599, 0.87638764]), 4: array([0.87282543, 0.86078608, 0.87788779]), 5: array([0.8815237 , 0.86348635, 0.88058806]), 6: array([0.87252549, 0.86258626, 0.87848785]), 7: array([0.87492501, 0.86288629, 0.8829883 ]), 8: array([0.8785243 , 0.86258626, 0.88058806]), 9: array([0.87282543, 0.85688569, 0.87938794]), 10: array([0.87492501, 0.85958596, 0.87968797])}
Wall time: 3.4 s


In [92]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='kd_tree', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.83803239, 0.83978398, 0.85358536]), 2: array([0.82543491, 0.82508251, 0.83858386]), 3: array([0.84853029, 0.84488449, 0.85508551]), 4: array([0.84913017, 0.84638464, 0.86378638]), 5: array([0.85122975, 0.84008401, 0.86648665]), 6: array([0.85302939, 0.84578458, 0.86228623]), 7: array([0.85722855, 0.84248425, 0.86618662]), 8: array([0.84973005, 0.84668467, 0.86348635]), 9: array([0.84673065, 0.84128413, 0.86528653]), 10: array([0.84883023, 0.83618362, 0.86168617])}
Wall time: 23.6 s


In [93]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='ball_tree', metric='euclidean', 
                    weights=False, test_block_size=10))

{1: array([0.83803239, 0.83978398, 0.85358536]), 2: array([0.82543491, 0.82508251, 0.83858386]), 3: array([0.84853029, 0.84488449, 0.85508551]), 4: array([0.84913017, 0.84638464, 0.86378638]), 5: array([0.85122975, 0.84008401, 0.86648665]), 6: array([0.85302939, 0.84578458, 0.86228623]), 7: array([0.85722855, 0.84248425, 0.86618662]), 8: array([0.84973005, 0.84668467, 0.86348635]), 9: array([0.84673065, 0.84128413, 0.86528653]), 10: array([0.84883023, 0.83618362, 0.86168617])}
Wall time: 18.2 s


In [94]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.29634073, 0.31173117, 0.31383138]), 2: array([0.29664067, 0.31323132, 0.31383138]), 3: array([0.30863827, 0.31653165, 0.31773177]), 4: array([0.31523695, 0.31893189, 0.32313231]), 5: array([0.31973605, 0.32223222, 0.31833183]), 6: array([0.32033593, 0.32283228, 0.32133213]), 7: array([0.31823635, 0.32133213, 0.32193219]), 8: array([0.31853629, 0.31953195, 0.31653165]), 9: array([0.31823635, 0.31953195, 0.31323132]), 10: array([0.31493701, 0.31713171, 0.31773177])}
Wall time: 1min 6s


In [95]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='my_own', metric='cosine', 
                    weights=True, test_block_size=10))

{1: array([0.089982  , 0.10861086, 0.10051005]), 2: array([0.08878224, 0.08520852, 0.07650765]), 3: array([0.08818236, 0.0840084 , 0.07920792]), 4: array([0.08458308, 0.08730873, 0.07980798]), 5: array([0.08428314, 0.09540954, 0.0840084 ]), 6: array([0.08458308, 0.0960096 , 0.08640864]), 7: array([0.08338332, 0.0960096 , 0.08520852]), 8: array([0.08428314, 0.09330933, 0.09210921]), 9: array([0.0839832 , 0.090009  , 0.08880888]), 10: array([0.08608278, 0.08820882, 0.09060906])}
Wall time: 31.3 s


In [96]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.83803239, 0.83978398, 0.85358536]), 2: array([0.83803239, 0.83978398, 0.85358536]), 3: array([0.85242951, 0.84488449, 0.86318632]), 4: array([0.85422915, 0.84668467, 0.86678668]), 5: array([0.85212957, 0.84488449, 0.86948695]), 6: array([0.85392921, 0.849985  , 0.86978698]), 7: array([0.85632873, 0.84818482, 0.8709871 ]), 8: array([0.85152969, 0.849985  , 0.86828683]), 9: array([0.85062987, 0.84428443, 0.86678668]), 10: array([0.84943011, 0.8409841 , 0.86678668])}
Wall time: 4.38 s


In [97]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='brute', metric='cosine', 
                    weights=True, test_block_size=10))

{1: array([0.85272945, 0.85208521, 0.86018602]), 2: array([0.85272945, 0.85208521, 0.86018602]), 3: array([0.87822436, 0.8619862 , 0.87968797]), 4: array([0.87762448, 0.8679868 , 0.88448845]), 5: array([0.88212358, 0.8679868 , 0.88418842]), 6: array([0.88122376, 0.8709871 , 0.88238824]), 7: array([0.88032394, 0.86828683, 0.88208821]), 8: array([0.88122376, 0.86858686, 0.88448845]), 9: array([0.88092382, 0.86468647, 0.88328833]), 10: array([0.87792442, 0.86408641, 0.88388839])}
Wall time: 3.49 s


In [98]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='kd_tree', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.83803239, 0.83978398, 0.85358536]), 2: array([0.83803239, 0.83978398, 0.85358536]), 3: array([0.85242951, 0.84488449, 0.86318632]), 4: array([0.85422915, 0.84668467, 0.86678668]), 5: array([0.85212957, 0.84488449, 0.86948695]), 6: array([0.85392921, 0.849985  , 0.86978698]), 7: array([0.85632873, 0.84818482, 0.8709871 ]), 8: array([0.85152969, 0.849985  , 0.86828683]), 9: array([0.85062987, 0.84428443, 0.86678668]), 10: array([0.84943011, 0.8409841 , 0.86678668])}
Wall time: 23.8 s


In [99]:
%%time
print(knn_cross_val_score(new_test_set, test_target, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'accuracy', None,  strategy='ball_tree', metric='euclidean', 
                    weights=True, test_block_size=10))

{1: array([0.83803239, 0.83978398, 0.85358536]), 2: array([0.83803239, 0.83978398, 0.85358536]), 3: array([0.85242951, 0.84488449, 0.86318632]), 4: array([0.85422915, 0.84668467, 0.86678668]), 5: array([0.85212957, 0.84488449, 0.86948695]), 6: array([0.85392921, 0.849985  , 0.86978698]), 7: array([0.85632873, 0.84818482, 0.8709871 ]), 8: array([0.85152969, 0.849985  , 0.86828683]), 9: array([0.85062987, 0.84428443, 0.86678668]), 10: array([0.84943011, 0.8409841 , 0.86678668])}
Wall time: 18.8 s
